# Assignments IF3170 - Artificial Intelligent
## Web Application for classifying hearth disease from data clinic 


## Phase A - Find Best Model

#### Group 3 - Unexpected
    - Rizki Alif Salman Alfarisy / 13516005
    - Jonathan Alvaro / 13516023
    - Joseph Salimin / 13516037
    - Kevin Leonardo Limitius / 13516049
    - Kevin Basuki / 13516071

### First of All.. Import All Library Needed

In [1]:
import numpy as np 
import pandas as pd 
import pickle
import itertools

from sklearn.model_selection import train_test_split, StratifiedKFold, train_test_split, KFold
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn import tree
from sklearn.impute import SimpleImputer

from sklearn import svm, datasets
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, classification_report, accuracy_score, confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.externals import joblib

from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

### Read Data From CSV

In [2]:
heart_train = pd.read_csv('tubes2_HeartDisease_train.csv')
heart_test = pd.read_csv('tubes2_HeartDisease_test.csv')

### Rename Column Names

In [3]:
from copy import deepcopy

test_columns = {
    'Column1': 'age',
    'Column2': 'sex',
    'Column3': 'chest_pain_type',
    'Column4': 'resting_blood_pressure',
    'Column5': 'serum_cholesterol',
    'Column6': 'fasting_blood_sugar',
    'Column7': 'resting_ecg',
    'Column8': 'max_heart_rate_achieved',
    'Column9': 'exercise_induced_angina',
    'Column10': 'st_depression',
    'Column11': 'peak_exercise_st_segment',
    'Column12': 'num_major_flourosopy',
    'Column13': 'thal'
}
# Create train columns
train_columns = test_columns.copy()
train_columns['Column14'] = 'heart_disease_diagnosis'

# Rename columns
heart_train = heart_train.rename(columns=train_columns)
heart_test = heart_test.rename(columns=test_columns)

### Data Analysis

In [4]:
print('Column data heart train')
pprint(heart_train.dtypes)
print()
print('Show heart train head')
pprint(heart_train.head())
print()
print('Find sum value undefined in each column')
heart_train.isna().sum()

Column data heart train
age                          int64
sex                          int64
chest_pain_type              int64
resting_blood_pressure      object
serum_cholesterol           object
fasting_blood_sugar         object
resting_ecg                 object
max_heart_rate_achieved     object
exercise_induced_angina     object
st_depression               object
peak_exercise_st_segment    object
num_major_flourosopy        object
thal                        object
heart_disease_diagnosis      int64
dtype: object

Show heart train head
   age  sex  chest_pain_type resting_blood_pressure serum_cholesterol  \
0   54    1                4                    125               216   
1   55    1                4                    158               217   
2   54    0                3                    135               304   
3   48    0                3                    120               195   
4   50    1                4                    120                 0   

  fasting_

age                         0
sex                         0
chest_pain_type             0
resting_blood_pressure      0
serum_cholesterol           0
fasting_blood_sugar         0
resting_ecg                 1
max_heart_rate_achieved     0
exercise_induced_angina     0
st_depression               0
peak_exercise_st_segment    0
num_major_flourosopy        0
thal                        0
heart_disease_diagnosis     0
dtype: int64

From above results, we can conclude that data given from CSV:
1. Not all data is numeric
2. There are some datas which value is '?'
3. There are also some datas which value is undefined (NaN)

**Our conclusion**: 

Results above can disturb the modeling process. Because of that, we need to do some pre-processing to ensure that there are little to no noise in the model.

## Dataframe Conversion to Numeric 

In [5]:
# Convert string to numeric
heart_train = heart_train.apply(pd.to_numeric, errors = 'coerce')
# Print data
print('Data type of columns after conversion')
print(heart_train.dtypes)
print()
# Show NaN count
print('Total value NaN after heart_train converted to numeric value')
print(heart_train.isna().sum())

Data type of columns after conversion
age                           int64
sex                           int64
chest_pain_type               int64
resting_blood_pressure      float64
serum_cholesterol           float64
fasting_blood_sugar         float64
resting_ecg                 float64
max_heart_rate_achieved     float64
exercise_induced_angina     float64
st_depression               float64
peak_exercise_st_segment    float64
num_major_flourosopy        float64
thal                        float64
heart_disease_diagnosis       int64
dtype: object

Total value NaN after heart_train converted to numeric value
age                           0
sex                           0
chest_pain_type               0
resting_blood_pressure       47
serum_cholesterol            24
fasting_blood_sugar          78
resting_ecg                   2
max_heart_rate_achieved      44
exercise_induced_angina      44
st_depression                49
peak_exercise_st_segment    262
num_major_flourosopy        51

#### Removing NaN Values

In the process above, first, we convert the data from object (string) to numeric value. We succeed in removing object value from dataframe. But there are also a problem, value which can not be convered to numeric data type will be converted to NaN and that problem can make dataframe hard to be processed.

One of the easiest way to remove NaN value is to remove row which contains NaN value in it. But that way is not really good and not feasible, since Column 12 has 514 rows which value is NaN and that means removing 514 rows which will reduce many data trainings. Another way is to replace NaN value with median. We choose median value rather than mean because median value is much more stable thant mean for irregular data (outliers).

For categorical data, for example in Column 3 and Column 7, it is better to replace NaN value with mode, since replacing the valie with median will result in creating meaningless value (not in that category). That's why, for Column 3, 6, 7, 11, and 13, we replace the value with mode.

Then, to reduce noise in data, we try to remove row which has more than 3 NaNs in its attribute columns

In [6]:
# Drop row with too many NaNs
heart_train = heart_train.dropna(thresh=10)

# # Fill NaN, median for continuous data, mode for categorical data
heart_train['thal'].fillna(heart_train['thal'].mode()[0], inplace=True)
heart_train['peak_exercise_st_segment'].fillna(heart_train['peak_exercise_st_segment'].mode()[0], inplace=True)
heart_train['chest_pain_type'].fillna(heart_train['chest_pain_type'].mode()[0], inplace=True)
heart_train['resting_ecg'].fillna(heart_train['resting_ecg'].mode()[0], inplace=True)
heart_train['fasting_blood_sugar'].fillna(heart_train['fasting_blood_sugar'].mode()[0], inplace=True)

imp = SimpleImputer(missing_values=np.nan, strategy='median')

c = heart_train.columns
heart_train = pd.DataFrame(imp.fit_transform(heart_train))
heart_train.columns = c

# Count NaN value
print('Total NaN Value')
print(heart_train.isna().sum())

Total NaN Value
age                         0
sex                         0
chest_pain_type             0
resting_blood_pressure      0
serum_cholesterol           0
fasting_blood_sugar         0
resting_ecg                 0
max_heart_rate_achieved     0
exercise_induced_angina     0
st_depression               0
peak_exercise_st_segment    0
num_major_flourosopy        0
thal                        0
heart_disease_diagnosis     0
dtype: int64


### Splitting of Categorical Data

For some categorical data, categories may be represented with value 1 - X. But, those values implicate that there is an ordering between categories (for example, 1 < 2 means that category 1 may be better or worse than category 2). Because of that, for some attribute, we dissolve it to create new X binary attribute 0 or 1 where X is total categories for that attribute. In our choice, we try to dissolve Column 11.

In [7]:
heart_train['upsloping'] = heart_train.apply(lambda row: row["peak_exercise_st_segment"] == 1, axis=1).astype(int)
heart_train['flat'] = heart_train.apply(lambda row: row["peak_exercise_st_segment"] == 2, axis=1).astype(int)
heart_train['downsloping'] = heart_train.apply(lambda row: row["peak_exercise_st_segment"] == 3, axis=1).astype(int)
heart_train = heart_train.drop('peak_exercise_st_segment', axis=1)

### Binarizing Data

Untuk data-data seperti resting blood pressure dan resting ecg, dilakukan binarisasi data (mengubah data menjadi nilai 1 atau 0). Hal ini dilakukan karena kedua kategori tersebut bisa disimplifikasi menjadi kategori sehat atau tidak (1 atau 0). Hal ini membuat data lebih simpel ketimbang sebelum dilakukan binarisasi, misalkan, untuk resting blood pressure, tanpa binarisasi, model-model yang dilatih harus bisa menemukan weight yang bisa memisahkan antara nilai pada range [120, 140] dan sisanya.

In [8]:
# Binarize resting_ecg, normal as 0 and everything else as 1
heart_train['resting_ecg'] = (heart_train['resting_ecg'] >= 1).astype(int)

In [9]:
# Binarize resting blood pressure to 0 for [120, 140] (healthy) and 1 otherwise
heart_train['resting_blood_pressure'] = ((heart_train['resting_blood_pressure'] > 140) | (heart_train['resting_blood_pressure'] < 120)).astype(int)

In [10]:
# Split data train
# heart_train_copy = heart_train.copy()
Y = heart_train['heart_disease_diagnosis']
X = heart_train.drop('heart_disease_diagnosis', axis = 1)

# Best so far
scaler = MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(X)

In [11]:
KF = StratifiedKFold(10, shuffle=True)

## Gaussian Naive-Bayes

In [12]:
sum_acc, sum_prec, sum_rec = 0, 0, 0
list_gnb_model = []

for trainidx, testidx in KF.split(X, Y):
    # Create GNB Model and test it
    gnb = GaussianNB()
    X_train, X_test = X[trainidx], X[testidx]
    Y_train, Y_test = Y[trainidx], Y[testidx]
    gnb.fit(X_train,Y_train)
    # Calculate accuraction, precision, and recall value
    accuration = metrics.accuracy_score(Y_test, gnb.predict(X_test))
    precision = metrics.precision_score(Y_test, gnb.predict(X_test), average="macro")
    recall = metrics.recall_score(Y_test, gnb.predict(X_test), average="macro")
    # Append it to list
    list_gnb_model.append((gnb, accuration, precision, recall))
    sum_acc += accuration
    sum_prec += precision
    sum_rec += recall

print("Average Accuration : {0:.4f}".format(sum_acc/10))
print("Average Precision : {0:.4f}".format(sum_prec/10))
print("Average Recall : {0:.4f}".format(sum_rec/10))

Average Accuration : 0.5684
Average Precision : 0.3610
Average Recall : 0.3524


## Decision Tree

In [13]:
sum_acc, sum_prec, sum_rec = 0, 0, 0
list_dt_model = []

for trainidx, testidx in KF.split(X, Y):
    # Create Tree Model and test it
    dt = tree.DecisionTreeClassifier()
    X_train, X_test = X[trainidx], X[testidx]
    Y_train, Y_test = Y[trainidx], Y[testidx]
    dt.fit(X_train,Y_train)
    # Calculate accuraction, precision, and recall value
    accuration = metrics.accuracy_score(Y_test, dt.predict(X_test))
    precision = metrics.precision_score(Y_test, dt.predict(X_test), average="macro")
    recall = metrics.recall_score(Y_test, dt.predict(X_test), average="macro")
    # Append it to list
    list_gnb_model.append((dt, accuration, precision, recall))
    sum_acc += accuration
    sum_prec += precision
    sum_rec += recall
    
print("Average Accuration : {0:.4f}".format(sum_acc/10))
print("Average Precision : {0:.4f}".format(sum_prec/10))
print("Average Recall : {0:.4f}".format(sum_rec/10))

Average Accuration : 0.5129
Average Precision : 0.3438
Average Recall : 0.3449


## KNN

In [14]:
sum_acc, sum_prec, sum_rec = 0, 0, 0
list_knn_model = []

for trainidx, testidx in KF.split(X, Y):
    # Create KNN Model and test it
    knn = KNeighborsClassifier(n_neighbors=44, weights='distance')
    X_train, X_test = X[trainidx], X[testidx]
    Y_train, Y_test = Y[trainidx], Y[testidx]
    knn.fit(X_train,Y_train)
    # Calculate accuraction, precision, and recall value
    accuration = metrics.accuracy_score(Y_test, knn.predict(X_test))
    precision = metrics.precision_score(Y_test, knn.predict(X_test), average="macro")
    recall = metrics.recall_score(Y_test, knn.predict(X_test), average="macro")
    # Append it to list
    list_knn_model.append((knn, accuration, precision, recall))
    sum_acc += accuration
    sum_prec += precision
    sum_rec += recall
    
print("Average Accuration : {0:.4f}".format(sum_acc/10))
print("Average Precision : {0:.4f}".format(sum_prec/10))
print("Average Recall : {0:.4f}".format(sum_rec/10))

Average Accuration : 0.5928
Average Precision : 0.3343
Average Recall : 0.3230


## MLP Classifier

In [18]:
sum_acc, sum_prec, sum_rec = 0, 0, 0
list_mlp_model = []

for trainidx, testidx in KF.split(X, Y):
    # Create MLP Model and test it
    mlp = MLPClassifier(hidden_layer_sizes=(2), solver='sgd', 
                        max_iter=1000, learning_rate_init=0.1, learning_rate='adaptive',
                        activation='identity')
    X_train, X_test = X[trainidx], X[testidx]
    Y_train, Y_test = Y[trainidx], Y[testidx]
    mlp.fit(X_train,Y_train)
    # Calculate accuraction, precision, and recall value
    accuration = metrics.accuracy_score(Y_test, mlp.predict(X_test))
    precision = metrics.precision_score(Y_test, mlp.predict(X_test), average="macro")
    recall = metrics.recall_score(Y_test, mlp.predict(X_test), average="macro")
    # Append it to list
    list_mlp_model.append((mlp, accuration, precision, recall))
    sum_acc += accuration
    sum_prec += precision
    sum_rec += recall
     
print("Average Accuration : {0:.4f}".format(sum_acc/10))
print("Average Precision : {0:.4f}".format(sum_prec/10))
print("Average Recall : {0:.4f}".format(sum_rec/10))

Average Accuration : 0.6022
Average Precision : 0.3783
Average Recall : 0.3601


## Save model

In [16]:
# Save model using Joblib
# Pilih salah satu yg terbaik

#joblib.dump(gnb, 'naivebayes.pkl')
#joblib.dump(tree, 'decisiontree.pkl')
joblib.dump(knn, 'knn.pkl')
#joblib.dump(mlp, 'mlp.pkl')

['knn.pkl']

## Load model

In [17]:
# Load model using Joblib
# Pilih salah satu yg terbaik

#gnb = joblib.load('naivebayes.pkl')
#tree = joblib.load('decisiontree.pkl')
knn = joblib.load('knn.pkl')
#mlp = joblib.load('mlp.pkl')

## Evaluasi Model